In [1]:
!pip install numpy

In [2]:
!pip install opencv-python

In [3]:
!pip install PySimpleGUI

In [4]:
import numpy as np
import cv2
import os.path
import PySimpleGUI as sg

In [5]:
version = "7 June 2020"

prototxt = r"colorization_deploy_v2.prototxt"
model = r"colorization_release_v2.caffemodel"
points = r"pts_in_hull.npy"

points = os.path.join(os.path.dirname("./"), points)
prototxt = os.path.join(os.path.dirname("./"), prototxt)
model = os.path.join(os.path.dirname("./"), model)
if not os.path.isfile(model):
    sg.popup_scrolled(
        "Missing model file",
        'You are missing the file "colorization_release_v2.caffemodel"',
        'Download it and place into your "model" folder',
        "You can download this file from this location:\n",
        r"https://www.dropbox.com/s/dx0qvhhp5hbcx7z/colorization_release_v2.caffemodel?dl=1",
    )
    exit()
net = cv2.dnn.readNetFromCaffe(prototxt, model)  # load model from disk
pts = np.load(points)

In [6]:
# add the cluster centers as 1x1 convolutions to the model
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = pts.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]

In [ ]:
import cv2
import numpy as np
import PySimpleGUI as sg

def colorize_image(image_filename=None, cv2_frame=None):
    # Load image or use provided frame
    image = cv2.imread(image_filename) if image_filename else cv2_frame
    if image is None:
        sg.popup_error("Error: Unable to load image.")
        return None, None

    # Convert to 3-channel grayscale if needed
    if len(image.shape) == 2 or image.shape[2] == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    elif image.shape[2] == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    # Colorization process
    scaled = image.astype("float32") / 255.0
    lab = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)
    resized = cv2.resize(lab, (224, 224))
    L = resized[:, :, 0]
    L -= 50

    net.setInput(cv2.dnn.blobFromImage(L))
    ab = net.forward()[0, :, :, :].transpose((1, 2, 0))
    ab = cv2.resize(ab, (image.shape[1], image.shape[0]))

    L = lab[:, :, 0]
    colorized = np.concatenate((L[:, :, np.newaxis], ab), axis=2)
    colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
    colorized = np.clip(colorized, 0, 1)
    colorized = (255 * colorized).astype("uint8")
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), colorized


def process_video(video_path, window, output_path=None):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        sg.popup_error("Error opening video file")
        return

    # Initialize video writer if needed
    writer = None
    if output_path:
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        fourcc = cv2.VideoWriter_fourcc(
            *"mp4v" if output_path.endswith(".mp4") else "XVID"
        )
        writer = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Process and display frames
        gray, colorized = colorize_image(cv2_frame=frame)
        if writer:
            writer.write(colorized)

        # Resize for display
        gray = cv2.resize(gray, (400, 400))
        colorized = cv2.resize(colorized, (400, 400))
        window["-IN-"].update(
            data=cv2.imencode(".png", cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR))[
                1
            ].tobytes()
        )
        window["-OUT-"].update(data=cv2.imencode(".png", colorized)[1].tobytes())

        # Handle events
        event, _ = window.read(timeout=10)
        if event in (None, "Exit", "-STOP-"):
            break

    cap.release()
    if writer:
        writer.release()


def process_webcam(window):
    ## Use generic video capture utillties from Open CV
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        sg.popup_error("Error opening webcam")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Process and display frames
        gray, colorized = colorize_image(cv2_frame=frame)

        # Resize for display
        gray = cv2.resize(gray, (400, 400))
        colorized = cv2.resize(colorized, (400, 400))
        window["-IN-"].update(
            data=cv2.imencode(".png", cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR))[
                1
            ].tobytes()
        )
        window["-OUT-"].update(data=cv2.imencode(".png", colorized)[1].tobytes())

        # Handle events
        event, _ = window.read(timeout=10)
        if event in (None, "Exit", "-STOP-"):
            break

    cap.release()


# GUI Layout
sg.theme("DarkBlue3")
layout = [
    [
        sg.Text("Input File:"),
        sg.Input(key="-IN FILE-"),
        sg.FileBrowse(),
        sg.Button("Colorize Photo", key="-PHOTO-"),
        sg.Button("Colorize Video", key="-VIDEO-"),
        sg.Button("Webcam", key="-WEBCAM-"),
        sg.Button("Stop", key="-STOP-"),
        sg.Button("Exit"),
    ],
    [
        sg.Column([[sg.Text("Original")], [sg.Image(key="-IN-", size=(400, 400))]]),
        sg.Column([[sg.Text("Colorized")], [sg.Image(key="-OUT-", size=(400, 400))]]),
    ],
    [
        sg.Button("Save Image", key="-SAVE-"),
        sg.Button("Save Video", key="-SAVE VIDEO-"),
    ],
]

window = sg.Window("Image/Video Colorizer", layout, resizable=True)

current_colorized = None

while True:
    event, values = window.read()
    # Based on the event value we can multiplex the functionalitites through the interface
    if event in (None, "Exit"):
        break

    if event == "-PHOTO-":
        if values["-IN FILE-"]:
            gray, colorized = colorize_image(image_filename=values["-IN FILE-"])
            if gray is not None and colorized is not None:
                current_colorized = colorized
                gray = cv2.resize(gray, (400, 400))
                colorized = cv2.resize(colorized, (400, 400))
                window["-IN-"].update(
                    data=cv2.imencode(".png", cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR))[
                        1
                    ].tobytes()
                )
                window["-OUT-"].update(
                    data=cv2.imencode(".png", colorized)[1].tobytes()
                )

    elif event == "-VIDEO-":
        if values["-IN FILE-"]:
            process_video(values["-IN FILE-"], window)

    elif event == "-WEBCAM-":
        process_webcam(window)

    elif event == "-SAVE-":
        if current_colorized is not None:
            filename = sg.popup_get_file(
                "Save Image", save_as=True, default_extension=".png"
            )
            if filename:
                cv2.imwrite(filename, current_colorized)

    elif event == "-SAVE VIDEO-":
        filename = sg.popup_get_file(
            "Save Video", save_as=True, default_extension=".mp4"
        )
        if filename and values["-IN FILE-"]:
            process_video(values["-IN FILE-"], window, filename)

window.close()